In [1]:
import re
import pandas as pd
from collections import Counter

df = pd.read_csv('all_en_lyrics_processed.csv')

In [2]:
# look at results where style is nan but genre not
# see if singular or list
# if list, how to define? only 688 that meet that criteria
# if only one genre, use that for MainGenre, if not, check style for regex or mapping

In [3]:
df

,Unnamed: 0,title,duration,album,date,artist,genre,style,id,country,source,lyrics,language,clean_lyrics,sentiment_score
0,0,El Tiempo,NaN,Fuerza Para Seguir,2007,Unidad 4,Rock,"Hardcore, Punk",3073827,Peru,discogs,3 ContributorsIris LyricsSo the song title and...,en,Ayo! Korgy! Shir! You know this 1337M0B Music ...,-0.895242
1,1,Poder,NaN,Fuerza Para Seguir,2007,Unidad 4,Rock,"Hardcore, Punk",3073827,Peru,discogs,2 Contributors2 Nigs United 4 West Compton (On...,en,"Two, three, get on it now I wanna sing, but it...",-0.986049
2,2,Fuckin' Aye Reloaded 4.0,178.0,Story Of Ohm,2007,Paniq,Electronic,"Breakbeat, Psy-Trance, Techno, Chiptune, Exper...",1151278,Germany,discogs,1 ContributorFuckin’ Aye Reloaded 4.0 LyricsYo...,en,You can't walk away now Hahaha...,0.998131
3,3,Flieg,NaN,10 Jaar Live / Simply Me,2007,Nádine,Pop,NaN,31648628,South Africa,discogs,1 ContributorLord Of The Flies Lyrics[Verse 1]...,en,I’ve led a crocodile smile life Grin to the fa...,-0.981931
4,4,You And I,NaN,10 Jaar Live / Simply Me,2007,Nádine,Pop,NaN,31648628,South Africa,discogs,1 ContributorYou and I Lyrics[Prehook: Jez]\n\...,en,You and I say A little fun and some Bombay Tha...,0.585116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10867,10867,Towns,286471.0,試合終了,2025-04-19,undersaken,NaN,NaN,194b90ee-5e39-4452-afa1-61b132d87f1a,NaN,musicbrainz,1 ContributorOld Town Road (Parody) Lyrics[Int...,en,"Oh, yeah Yeah, I'm gonna take my hoes to the h...",-0.989772
10868,10868,FISHSCALE FRIDAY,58656.0,HEELS HAVE EYES,2025-04-18,Westside Gunn,NaN,NaN,c5f5af4d-bef0-4ad1-95fe-6646da8c175a,NaN,musicbrainz,15 ContributorsTranslationsPortuguêsFISHSCALE ...,en,"Awesome God Awesome, awesome, awesome, awesome...",0.999152
10869,10869,EINSTEIN KITCHEN,82392.0,HEELS HAVE EYES,2025-04-18,Westside Gunn,NaN,NaN,9062640c-b3fd-47f8-8b1d-ac523b49256e,NaN,musicbrainz,17 ContributorsTranslationsPortuguêsEINSTEIN K...,en,"Yeah, yeah (Griselda by Fashion Rebels) Ayo, t...",-0.996129
10870,10870,GORO,128976.0,HEELS HAVE EYES,2025-04-18,Westside Gunn,NaN,NaN,ff125a35-414f-4622-828d-625387fbe389,NaN,musicbrainz,15 ContributorsTranslationsPortuguêsGORO Lyric...,en,"Brrt (La música de Harry Fraud) Ayo, ran up in...",-0.997895


In [4]:
df['genre'].value_counts()

genre
Rock                                                                                  508
Rock, Pop                                                                             299
Hip Hop                                                                               196
Electronic                                                                            162
Rock, Blues                                                                            82
Pop                                                                                    56
Folk, World, & Country                                                                 54
Funk / Soul, Pop                                                                       39
Pop, Folk, World, & Country                                                            39
Pop, Children's                                                                        32
Classical, Folk, World, & Country                                                      32
Elec

In [5]:
#subset2 = df[df['genre'].apply(lambda x: isinstance(x, str) and ',' in x) & df['style'].isna()]
#subset2

In [6]:
#subset = df[df['genre'].notna() & df['style'].isna()]
#subset

In [7]:
#subset2.shape

In [8]:
#subset.shape

In [9]:
def assign_single_genre(df):
    """Process the genre column to assign a single genre per record using regex and majority selection."""
    
    # Discog main genres
    main_genres = [
        'Rock', 'Electronic', 'Pop', 'Folk', 'World', 'Country', 'Jazz', 
        'Funk / Soul', 'Classical', 'Hip Hop', 'Latin', 'Stage & Screen', 
        'Reggae', 'Blues', 'Non-Music', "Children's", 'Brass & Military'
    ]
    
    def clean_and_extract_genres(genre_str):
        if pd.isna(genre_str) or genre_str == '':
            return []

        genre_clean = re.sub(r'[^\w\s,&/-]', '', str(genre_str))

        potential_genres = re.split(r'[,&/]+', genre_clean)
        
        valid_genres = []
        for genre in potential_genres:
            genre = genre.strip()
            if not genre:
                continue

            for main_genre in main_genres:
                if genre.lower() == main_genre.lower():
                    valid_genres.append(main_genre)
                    break
            else:
                for main_genre in main_genres:
                    if main_genre.lower() in genre.lower():
                        valid_genres.append(main_genre)
                        break
        
        return valid_genres
    
    def find_genre_in_style(style_str):
        if pd.isna(style_str) or style_str == '':
            return []
        
        style_lower = str(style_str).lower()
        found_genres = []
        
        for main_genre in main_genres:
            # Handle special cases
            if main_genre == 'Funk / Soul':
                if 'funk' in style_lower or 'soul' in style_lower:
                    found_genres.append(main_genre)
            elif main_genre == 'Hip Hop':
                if 'hip hop' in style_lower or 'hip-hop' in style_lower or 'rap' in style_lower:
                    found_genres.append(main_genre)
            elif main_genre == 'Stage & Screen':
                if 'soundtrack' in style_lower or 'musical' in style_lower or 'score' in style_lower:
                    found_genres.append(main_genre)
            elif main_genre == 'Brass & Military':
                if 'brass' in style_lower or 'military' in style_lower or 'marches' in style_lower:
                    found_genres.append(main_genre)
            elif main_genre == "Children's":
                if "children" in style_lower or "nursery" in style_lower:
                    found_genres.append(main_genre)
            else:
                if main_genre.lower() in style_lower:
                    found_genres.append(main_genre)
        
        return found_genres
    
    def get_best_genre(genres_from_genre, genres_from_style):
        all_genres = genres_from_genre + genres_from_style
        
        if not all_genres:
            return 'Unknown'  # Default fallback - check to see how many and if dropping 

        genre_counts = Counter(all_genres)

        return genre_counts.most_common(1)[0][0]

    df_result = df.copy()

    def assign_genre_row(row):
        genres_from_genre = clean_and_extract_genres(row['genre'])
        genres_from_style = find_genre_in_style(row.get('style', ''))
        return get_best_genre(genres_from_genre, genres_from_style)
    
    df_result['genre'] = df_result.apply(assign_genre_row, axis=1)
    
    return df_result

In [10]:
# Example usage:
df_processed = assign_single_genre(df)
print(df_processed['genre'].value_counts())

genre
Unknown        9228
Rock            857
Pop             242
Hip Hop         202
Electronic      198
Folk             55
Classical        33
Jazz             26
Blues            12
Funk / Soul      11
Non-Music         6
Reggae            2
Name: count, dtype: int64
